# Make plots of binned-by-column-RH fields

In [1]:
import xarray as xr
import glob
import numpy as np
import proplot as pplt
%config InlineBackend.figure_format = 'retina'
pplt.rc.update({'font.size':12})
import warnings
warnings.filterwarnings('ignore')

In [2]:
# where are files located
fpath = 

cases = ['nz_32','nz_64','nz_128']

# where to save files
figdir = '/jet/home/ajenney/repos/conv_agg_vres/figs/'

In [4]:
# Add files to dslist manually because of weird timeframe dimension
# and multiple files for nz_128

dslist = []

for case in cases:
    dslist.append(xr.open_dataset(glob.glob(fpath + 'crh_binned_vars_nz_' + 
                                            case + '_figure_timeframes.nc')[0],
                                  decode_cf=False))

In [6]:
# Read in some variables
timeframes = dslist[0].timeframe.values
bincenter = dslist[0].percentile_center.values

# Reformat timeframes string so its formatted with leading zeros
for ii,timeframe in enumerate(timeframes):
    # Find the index of the dash
    idash = timeframe.index('-')
    start = int(timeframe[:idash])
    stop = int(timeframe[idash+1:])
    
    timeframes[ii] = '{:03d}'.format(start) + '-{:03d}'.format(stop)
    
timeframes

array(['000-025', '025-050', '100-125'], dtype=object)

In [7]:
# Plot one timeframe at a time

# Plot specifications
top='RdBu_r'; bottom='Cubehelix'
qnlevels=[0.003,0.006,0.009,0.012]
dpsi = 0.0010
psilevels = np.arange(-0.0110, 0.0110+dpsi, dpsi)

for itime,timeframe in enumerate(timeframes):

    fig,axs = pplt.subplots(ncols=3,nrows=2,refwidth=2,refheight=3)

    axs.format(xlabel='Column relative humidity percentile',xlim=(0,100),
               xticks=25,ylabel='z (km)',ylim=(0,13),
               abc='a',abcloc='l',suptitle='Days '+ timeframe)

    for icase,case in enumerate(cases):
        # Top row
        qradplots = axs[icase].contourf(
                    bincenter,
                    dslist[icase].z/1000,
                    np.transpose(dslist[icase].Qr.isel(timeframe=itime).values),
                    cmap=top,levels=20,
                    vmin=-1.5,vmax=1.5,extend='both')
        axs[icase].contour(bincenter,dslist[icase].z/1000,
                           np.transpose(dslist[icase].psi.isel(timeframe=itime).values),
                           color='k',linewidth=0.8,levels=psilevels)
        axs[icase].contour(bincenter,dslist[icase].z/1000,
                       np.transpose(dslist[icase].qn.isel(timeframe=itime).values),
                       color='w',levels=qnlevels,linewidth=0.8)
        axs[icase].format(titleloc='l',title=case+', $Q_{R}$') 

        # Bottom row
        fmseplots = axs[icase+3].contourf(
                    bincenter,
                    dslist[icase].z/1000,
                    np.transpose(dslist[icase].fmse.isel(timeframe=itime).values)/1000,
                    cmap=bottom,levels=21,
                    vmin=315,vmax=335,extend='both')
        axs[icase+3].contour(bincenter,dslist[icase].z/1000,
                           np.transpose(dslist[icase].psi.isel(timeframe=itime).values),
                           color='k',linewidth=0.8,levels=psilevels)
        axs[icase+3].contour(bincenter,dslist[icase].z/1000,
                       np.transpose(dslist[icase].qn.isel(timeframe=itime).values),
                       color='w',levels=qnlevels,linewidth=0.8)
        axs[icase+3].format(titleloc='l',
                        title=case+ r', $\langle h_{\mathrm{f}} \rangle$')

    # Colorbars
    axs[2].colorbar(qradplots,loc='r',label='Radiative heating rate (K day$^{-1}$)')
    axs[5].colorbar(fmseplots,loc='r',label='Frozen moist static energy (kJ m$^{-2}$)')

    fig.savefig(figdir + 'by_crh_d' + timeframe + '.png', dpi=300)
    pplt.close('all')